In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Daily Scheduler — aligned with backtest pipeline (VOLAᵣ → slots → MVO → deploy cap)

ENTRY (D0 signal, D1 execution)
  • EMA10 > EMA20
  • Two strict green candles (D-1, D0) AND D0’s O/H/L/C > D-1’s O/H/L/C
  • ≥1 confirm: RSI>50 | MACD>Signal | ADX>20 & +DI>-DI | Close>SMA50 | Close>BBmid
  • Optional 52w filter: Close ≥ within_pct_of_52w_high × 52w-high

PORTFOLIO (matches backtest)
  1) Rank survivors by VOLAᵣ vs benchmark (252d).
  2) slots = max_concurrent_positions – open_positions (from positions.csv)
  3) pick = head(min(top_k_daily, slots))
  4) MVO (long-only) on 252d returns → weights
  5) deployable cash = deploy_cash_frac × current_cash
     - current_cash = initial_capital – MTM invested (or override CURRENT_CASH_INR env)

EXIT checks for open positions (latest completed bar)
  • HSL (Low ≤ SL) → plan SELL@SL
  • TP  (High ≥ TP) → plan SELL@TP
  • Trend reverse (EMA10<EMA20) → plan SELL@next open

OUTPUTS
  outputs/actionable_buys_ranked_YYYYMMDD.csv
  outputs/actionable_buys_selected_YYYYMMDD.csv (mvo_weight, alloc_inr, est_shares)
  outputs/actionable_sells_YYYYMMDD.csv

TELEGRAM
  • Set TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID; toggle via TELEGRAM_ENABLED=0/1 or CFG.telegram_enabled
"""

import os, sys, math, logging, warnings
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
    import requests
except Exception as e:
    print("Missing deps. Install:\n  pip install yfinance requests pandas numpy", file=sys.stderr)
    raise

# ---------------- Logging ----------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("daily_scheduler")

# ---------------- Config -----------------
@dataclass
class Config:
    # Universe
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = "nifty500.txt"   # one symbol per line

    # Data
    lookback_days: int = 400
    out_dir: str = "outputs"

    # Positions ledger (YOU maintain this)
    positions_csv: str = "positions.csv"

    # Strategy (aligned to backtester)
    ema_fast: int = 10
    ema_slow: int = 20
    rsi_len: int = 14
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9
    bb_len: int = 20
    bb_std: float = 2.0
    sma_confirm_len: int = 50
    adx_len: int = 14
    adx_min: float = 20.0

    # Filters
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50  # 50%

    # Exits (for SELL evaluation only)
    stop_loss_pct: float = 0.10
    target_pct: float = 0.10

    # Portfolio controls (aligned to your backtest)
    initial_capital: float = 500_000.0
    deploy_cash_frac: float = 0.30
    max_concurrent_positions: int = 5
    volar_lookback: int = 252
    top_k_daily: int = 300

    # Telegram
    telegram_enabled: bool = True  # toggle on/off
    telegram_bot_token: Optional[str] = None
    telegram_chat_id: Optional[str] = None

CFG = Config()

# ---------------- Utils ------------------
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_stamp():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d")

def load_universe(cfg: Config) -> List[str]:
    if cfg.static_symbols:
        syms = [s.strip().upper() for s in cfg.static_symbols if s.strip()]
    elif cfg.static_symbols_path and os.path.exists(cfg.static_symbols_path):
        with open(cfg.static_symbols_path, "r") as f:
            syms = [line.strip().upper() for line in f if line.strip()]
    else:
        raise ValueError("Provide static_symbols or static_symbols_path for the universe.")
    out = []
    for s in syms:
        if not s.endswith(".NS") and not s.startswith("^"):
            s = f"{s}.NS"
        out.append(s)
    seen, uniq = set(), []
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], days: int) -> Dict[str, pd.DataFrame]:
    end = pd.Timestamp.today(tz="Asia/Kolkata").normalize()
    start = end - pd.Timedelta(days=days*1.5)
    start, end_s = start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")
    data = {}
    for t in tickers:
        try:
            df = yf.download(t, start=start, end=end_s, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            data[t] = df
        except Exception:
            continue
    return data

# ------------- Indicators ----------------
def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def sma(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0).rolling(length).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)

def macd(series: pd.Series, fast=12, slow=26, signal=9):
    ef = series.ewm(span=fast, adjust=False, min_periods=slow).mean()
    es = series.ewm(span=slow, adjust=False, min_periods=slow).mean()
    line = ef - es
    sig = line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    hist = line - sig
    return line, sig, hist

def _true_range(high, low, prev_close):
    return pd.concat([(high - low).abs(), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)

def dmi_adx(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14):
    ph, pl, pc = high.shift(1), low.shift(1), close.shift(1)
    up, dn = high - ph, pl - low
    plus_dm  = up.where((up > dn) & (up > 0), 0.0)
    minus_dm = dn.where((dn > up) & (dn > 0), 0.0)
    tr = _true_range(high, low, pc)
    a = 1.0 / length
    atr = tr.ewm(alpha=a, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=a, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=a, adjust=False, min_periods=length).mean() / atr)
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_series = dx.ewm(alpha=a, adjust=False, min_periods=length).mean()
    return plus_di, minus_di, adx_series

def bollinger(series: pd.Series, length=20, nstd=2.0):
    mid = series.rolling(length).mean()
    sd  = series.rolling(length).std(ddof=0)
    upper = mid + nstd * sd
    lower = mid - nstd * sd
    return mid, upper, lower

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    d = df.copy()
    d["ema_fast"] = ema(d["Close"], cfg.ema_fast)
    d["ema_slow"] = ema(d["Close"], cfg.ema_slow)
    d["sma50"]    = sma(d["Close"], cfg.sma_confirm_len)
    d["rsi"]      = rsi(d["Close"], cfg.rsi_len)
    macd_line, macd_signal, macd_hist = macd(d["Close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    d["macd_line"], d["macd_signal"], d["macd_hist"] = macd_line, macd_signal, macd_hist
    d["bb_mid"], d["bb_up"], d["bb_low"] = bollinger(d["Close"], cfg.bb_len, cfg.bb_std)
    d["+DI"], d["-DI"], d["ADX"] = dmi_adx(d["High"], d["Low"], d["Close"], cfg.adx_len)
    d["high_52w"] = d["Close"].rolling(cfg.filter_52w_window).max()
    d["avg_vol_20"] = d["Volume"].rolling(20).mean()
    return d.dropna()

# ------------- Entry/Exit logic ----------
def two_green_strict(prev_row: pd.Series, row: pd.Series) -> bool:
    g1 = prev_row["Close"] > prev_row["Open"]
    g2 = row["Close"] > row["Open"]
    strictly_above = (row["Open"]  > prev_row["Open"]) and \
                     (row["High"]  > prev_row["High"]) and \
                     (row["Low"]   > prev_row["Low"])  and \
                     (row["Close"] > prev_row["Close"])
    return bool(g1 and g2 and strictly_above)

def confirmation_any(row: pd.Series, cfg: Config) -> Tuple[bool, str]:
    checks = [
        ("RSI>50", row["rsi"] > 50.0),
        ("MACD>Signal", row["macd_hist"] > 0.0),
        ("ADX>20 & +DI>-DI", (row["ADX"] > cfg.adx_min) and (row["+DI"] > row["-DI"])),
        ("Close>SMA50", row["Close"] > row["sma50"]),
        ("Close>BBmid", row["Close"] > row["bb_mid"]),
    ]
    passed = [name for name, ok in checks if ok]
    return (len(passed) >= 1, ", ".join(passed) if passed else "none")

def volar_score(series: pd.Series, bench: pd.Series, lookback: int) -> float:
    r_s = series.pct_change().dropna().iloc[-lookback:]
    r_b = bench.pct_change().dropna().iloc[-lookback:]
    common = pd.concat([r_s, r_b], axis=1).dropna()
    if common.shape[0] < max(20, int(0.4*lookback)):
        return 0.0
    excess = common.iloc[:,0] - common.iloc[:,1]
    vol = common.iloc[:,0].std(ddof=0)
    return 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))

# --------------- Telegram ----------------
def send_telegram(text: str, cfg: Config):
    # toggle via config or env TELEGRAM_ENABLED=0/1
    env_toggle = os.getenv("TELEGRAM_ENABLED")
    if env_toggle is not None:
        try:
            enabled = bool(int(env_toggle))
        except Exception:
            enabled = cfg.telegram_enabled
    else:
        enabled = cfg.telegram_enabled

    if not enabled:
        log.info("Telegram disabled. Skipping message.")
        return

    token = cfg.telegram_bot_token or os.getenv("TELEGRAM_BOT_TOKEN", "")
    chat  = cfg.telegram_chat_id or os.getenv("TELEGRAM_CHAT_ID", "")
    if not token or not chat:
        log.warning("Telegram not configured (missing token/chat). Skipping.")
        return
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    try:
        requests.post(url, data={"chat_id": chat, "text": text, "parse_mode": "Markdown"}, timeout=20)
    except Exception as e:
        log.warning("Telegram send failed: %s", e)

# ------------- Positions ledger ----------
def ensure_positions_csv(path: str):
    if os.path.exists(path): return
    cols = ["ticker","entry_date","entry_price","shares","sl_pct","tp_pct","sl_price","tp_price","notes","status"]
    pd.DataFrame(columns=cols).to_csv(path, index=False)
    log.info("Created blank %s. Add your taken trades as rows.", path)

def load_positions(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame(columns=["ticker","entry_date","entry_price","shares","sl_pct","tp_pct","sl_price","tp_price","notes","status"])
    df = pd.read_csv(path)
    for c in ["sl_pct","tp_pct"]:
        if c not in df.columns: df[c] = np.nan
    for c in ["sl_price","tp_price"]:
        if c not in df.columns: df[c] = np.nan
    if "status" not in df.columns:
        df["status"] = "open"
    # normalize tickers
    df["ticker"] = df["ticker"].astype(str).str.upper().apply(lambda s: s if s.endswith(".NS") or s.startswith("^") else f"{s}.NS")
    return df

def finalize_sl_tp(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    df = df.copy()
    df["sl_pct"] = df["sl_pct"].fillna(cfg.stop_loss_pct)
    df["tp_pct"] = df["tp_pct"].fillna(cfg.target_pct)
    df["sl_price"] = df["sl_price"].fillna(df["entry_price"] * (1 - df["sl_pct"]))
    df["tp_price"] = df["tp_price"].fillna(df["entry_price"] * (1 + df["tp_pct"]))
    return df

# --------------- MVO tools ---------------
def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu)
    eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)

    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w = sol[:n]
            except np.linalg.LinAlgError:
                w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0: return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]
            o = np.ones(len(idx))
            m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w_sub = sol[:len(idx)]
            except np.linalg.LinAlgError:
                w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx] = w_sub
            return w

    best_w = np.full(n, 1.0/n)
    best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool)
        w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            if not (w < 0).any(): break
            worst = np.argmin(w)
            active[worst] = False
        if w is None: continue
        w = np.clip(w, 0, None)
        if w.sum() <= 0: continue
        w = w / w.sum()
        mu_p = float(mu @ w)
        vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8: continue
        sr = mu_p / vol_p
        if sr > best_sr:
            best_sr = sr
            best_w = w.copy()
    return best_w

def estimate_current_cash(pos_open: pd.DataFrame, data_map: Dict[str,pd.DataFrame], cfg: Config) -> float:
    # Prefer an explicit override (captures realized P&L)
    env_cash = os.getenv("CURRENT_CASH_INR", "")
    if env_cash:
        try:
            return max(0.0, float(env_cash))
        except ValueError:
            pass
    # Approx: initial_capital minus MTM of holdings
    invested = 0.0
    for _, pos in pos_open.iterrows():
        t = pos["ticker"]; sh = float(pos.get("shares", 0) or 0)
        df = data_map.get(t)
        if df is None or df.empty: continue
        last_close = float(df["Close"].iloc[-1])
        invested += sh * last_close
    return max(0.0, cfg.initial_capital - invested)

# --------------- Scanners ----------------
def scan_buys(universe: List[str], data_map: Dict[str,pd.DataFrame], cfg: Config,
              bench_df: Optional[pd.DataFrame]) -> pd.DataFrame:
    rows = []
    for t in universe:
        df = data_map.get(t)
        if df is None or df.empty: 
            continue
        d = compute_indicators(df, cfg)
        if d.empty or len(d) < 60: 
            continue
        idx = list(d.index)
        if len(idx) < 2: 
            continue
        dt, prev_dt = idx[-1], idx[-2]
        row, prev_row = d.loc[dt], d.loc[prev_dt]

        ema_ok = row["ema_fast"] > row["ema_slow"]
        two_ok = two_green_strict(prev_row, row)
        conf_ok, conf_str = confirmation_any(row, cfg)
        if not (ema_ok and two_ok and conf_ok):
            continue

        # 52w filter (as in backtest pipeline)
        high_52w = float(row["high_52w"]); close_val = float(row["Close"])
        if not (high_52w > 0 and close_val >= cfg.within_pct_of_52w_high * high_52w):
            continue

        rows.append({
            "ticker": t, "signal_date": dt, "close": close_val,
            "ema10": float(row["ema_fast"]), "ema20": float(row["ema_slow"]),
            "rsi": float(row["rsi"]), "macd_hist": float(row["macd_hist"]), "adx": float(row["ADX"]),
            "pdi": float(row["+DI"]), "mdi": float(row["-DI"]),
            "sma50": float(row["sma50"]), "bb_mid": float(row["bb_mid"]),
            "confirm": conf_str,
            "sig_reason": f"EMA10>EMA20 & 2xGreenStrict; confirms: {conf_str}",
            "high_52w": high_52w,
        })
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # VOLAᵣ ranking (252d) — same selection signal as backtest
    if bench_df is not None and not bench_df.empty:
        bench_ser = bench_df["Close"]
        df["volar"] = [
            volar_score(data_map[t]["Close"], bench_ser, cfg.volar_lookback)
            for t in df["ticker"]
        ]
        df = df.sort_values("volar", ascending=False)

    if cfg.top_k_daily and len(df) > cfg.top_k_daily:
        df = df.head(cfg.top_k_daily)
    return df.reset_index(drop=True)

def scan_sells(positions_open: pd.DataFrame, data_map: Dict[str,pd.DataFrame], cfg: Config) -> pd.DataFrame:
    if positions_open.empty:
        return pd.DataFrame(columns=["ticker","signal_date","plan","reason","last_close","entry_price","sl_price","tp_price"])
    rows = []
    for _, pos in positions_open.iterrows():
        t = pos["ticker"]
        df = data_map.get(t)
        if df is None or df.empty:
            continue
        d = compute_indicators(df, cfg)
        if d.empty: 
            continue
        idx = list(d.index)
        if len(idx) < 2: 
            continue
        dt, prev_dt = idx[-1], idx[-2]
        row, prev_row = d.loc[dt], d.loc[prev_dt]

        entry_px = float(pos["entry_price"])
        sl_price = float(pos["sl_price"]) if not pd.isna(pos["sl_price"]) else entry_px*(1-cfg.stop_loss_pct)
        tp_price = float(pos["tp_price"]) if not pd.isna(pos["tp_price"]) else entry_px*(1+cfg.target_pct)

        day_low  = float(row["Low"])
        day_high = float(row["High"])
        last_close = float(row["Close"])

        # Priority like backtest: if both touch, treat as TP (optimistic)
        hit = None; plan_price = None
        if (day_low <= sl_price) and (day_high >= tp_price):
            hit, plan_price = "target", tp_price
        elif (day_low <= sl_price):
            hit, plan_price = "stop", sl_price
        elif (day_high >= tp_price):
            hit, plan_price = "target", tp_price
        else:
            if row["ema_fast"] < row["ema_slow"]:
                hit, plan_price = "trend_reverse", None

        if hit is None:
            continue

        pnl_pct_est = (plan_price / entry_px - 1.0) * 100.0 if plan_price is not None else (last_close / entry_px - 1.0) * 100.0

        if hit == "stop":
            reason = (f"StopLoss hit ({pos.get('sl_pct', cfg.stop_loss_pct)*100:.1f}%). "
                      f"today H/L={day_high:.2f}/{day_low:.2f}, SL={sl_price:.2f}; "
                      f"plan=Exit@SL, ret≈{pnl_pct_est:.2f}%")
            plan = f"SELL @ {sl_price:.2f}"
        elif hit == "target":
            reason = (f"TakeProfit hit ({pos.get('tp_pct', cfg.target_pct)*100:.1f}%). "
                      f"today H/L={day_high:.2f}/{day_low:.2f}, TP={tp_price:.2f}; "
                      f"plan=Exit@TP, ret≈{pnl_pct_est:.2f}%")
            plan = f"SELL @ {tp_price:.2f}"
        else:
            crossdown = (prev_row["ema_fast"] >= prev_row["ema_slow"]) and (row["ema_fast"] < row["ema_slow"])
            cx = " cross↓" if crossdown else ""
            reason = (f"Trend reverse: EMA10{cx} below EMA20 (10={row['ema_fast']:.2f}, 20={row['ema_slow']:.2f}); "
                      f"plan=Exit@next open, ret≈{pnl_pct_est:.2f}% (vs last close)")
            plan = "SELL @ next open"

        rows.append({
            "ticker": t, "signal_date": dt, "plan": plan, "reason": reason,
            "last_close": last_close, "entry_price": entry_px, "sl_price": sl_price, "tp_price": tp_price
        })
    return pd.DataFrame(rows).sort_values(["ticker"]).reset_index(drop=True)

# --------------- MAIN --------------------
def main():
    ensure_dirs(CFG.out_dir)
    ensure_positions_csv(CFG.positions_csv)

    # Universe & positions
    universe = load_universe(CFG)
    pos = load_positions(CFG.positions_csv)
    pos = finalize_sl_tp(pos, CFG)
    pos_open = pos[pos["status"].astype(str).str.lower().eq("open")].copy()

    # Pull data for union(universe, open positions, candidate benchmarks)
    bench_try = ("^CRSLDX","^CNX500","^NSE500","^NIFTY500","^BSE500","^NSEI")
    extra = [t for t in pos_open["ticker"].unique().tolist() if t not in universe]
    tickers = list(dict.fromkeys(universe + extra + list(bench_try)))
    log.info("Fetching data for %d tickers...", len(tickers))
    data_map = fetch_prices(tickers, CFG.lookback_days)

    # Benchmark for VOLAᵣ
    bench_tkr = next((t for t in bench_try if t in data_map), None)
    bench_df = data_map.get(bench_tkr) if bench_tkr else None
    if bench_tkr: log.info("Benchmark for ranking: %s", bench_tkr)

    # BUY scan (D0 signals for D1 execution)
    buys_ranked = scan_buys(universe, data_map, CFG, bench_df if bench_tkr else None)

    # SELL scan on open positions
    sells = scan_sells(pos_open, data_map, CFG)

    # Selection capacity (like backtest)
    open_count = pos_open.shape[0]
    slots = max(0, CFG.max_concurrent_positions - open_count)
    if slots > 0 and not buys_ranked.empty:
        buys_sel = buys_ranked.head(min(slots, len(buys_ranked))).copy()
    else:
        buys_sel = buys_ranked.iloc[0:0].copy()

    # MVO sizing over selected names
    if not buys_sel.empty:
        names = buys_sel["ticker"].tolist()
        rets = []
        for t in names:
            ser = data_map[t]["Close"].pct_change().dropna().iloc[-CFG.volar_lookback:]
            rets.append(ser.rename(t))
        R = pd.concat(rets, axis=1).dropna()
        if R.empty or R.shape[0] < max(20, int(0.4*CFG.volar_lookback)) or R.shape[1] == 0:
            weights = np.full(len(names), 1.0/len(names))
        else:
            mu = R.mean().values
            Sigma = R.cov().values
            weights = markowitz_long_only(mu, Sigma)
        weights = weights / weights.sum()

        current_cash = estimate_current_cash(pos_open, data_map, CFG)
        deploy_cash  = max(0.0, current_cash) * CFG.deploy_cash_frac

        buys_sel["mvo_weight"] = weights
        buys_sel["alloc_inr"]  = buys_sel["mvo_weight"] * deploy_cash
        buys_sel["est_shares"] = (buys_sel["alloc_inr"] / buys_sel["close"]).apply(
            lambda x: int(math.floor(x)) if np.isfinite(x) else 0
        )

    # Write outputs
    stamp = today_stamp()
    ranked_path = os.path.join(CFG.out_dir, f"actionable_buys_ranked_{stamp}.csv")
    select_path = os.path.join(CFG.out_dir, f"actionable_buys_selected_{stamp}.csv")
    sells_path  = os.path.join(CFG.out_dir, f"actionable_sells_{stamp}.csv")

    if not buys_ranked.empty: buys_ranked.to_csv(ranked_path, index=False)
    if not buys_sel.empty:    buys_sel.to_csv(select_path, index=False)
    if not sells.empty:       sells.to_csv(sells_path, index=False)

    # Telegram summaries (toggle-able)
    if (buys_sel.empty and sells.empty):
        send_telegram("📭 No new BUY or SELL signals today.", CFG)
    else:
        if not buys_sel.empty:
            lines = [f"📈 *BUY (selected & MVO-sized)* — slots={slots}, deploy={CFG.deploy_cash_frac*100:.0f}% of cash"]
            for _, r in buys_sel.iterrows():
                alloc = f"₹{r['alloc_inr']:.0f}" if np.isfinite(r.get("alloc_inr", np.nan)) else "n/a"
                wtxt  = f"{r['mvo_weight']*100:.1f}%" if np.isfinite(r.get("mvo_weight", np.nan)) else "n/a"
                lines.append(f"• {r['ticker']} — {wtxt}, alloc≈{alloc}, est_shares≈{int(r['est_shares'])} — {r['sig_reason']}")
            send_telegram("\n".join(lines), CFG)
        if not sells.empty:
            lines = [f"📉 *SELL signals* ({len(sells)})"]
            for _, r in sells.iterrows():
                lines.append(f"• {r['ticker']} — {r['plan']}\n   {r['reason']}")
            send_telegram("\n".join(lines), CFG)

    log.info("Written: %s%s%s",
             ranked_path if not buys_ranked.empty else "(no ranked buys)",
             f", {select_path}" if not buys_sel.empty else "",
             f", {sells_path}" if not sells.empty else "")

if __name__ == "__main__":
    # Allow env overrides
    CFG.telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN", None)
    CFG.telegram_chat_id   = os.getenv("TELEGRAM_CHAT_ID", None)
    env_toggle = os.getenv("TELEGRAM_ENABLED")
    if env_toggle is not None:
        try: CFG.telegram_enabled = bool(int(env_toggle))
        except Exception: pass
    main()


2025-10-27 19:04:48 | INFO | Fetching data for 506 tickers...
2025-10-27 19:06:04 | ERROR | 
1 Failed download:
2025-10-27 19:06:04 | ERROR | ['HONAUT.NS']: Timeout('Failed to perform, curl: (28) Operation timed out after 10001 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
2025-10-27 19:07:23 | ERROR | 
1 Failed download:
2025-10-27 19:07:23 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-27 19:07:24 | ERROR | 
1 Failed download:
2025-10-27 19:07:24 | ERROR | ['^NSE500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-27 19:07:25 | ERROR | 
1 Failed download:
2025-10-27 19:07:25 | ERROR | ['^NIFTY500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-27 19:07:25 | ERROR | 
1 Failed download:
2025-10-27 19:07:25 | ERROR | ['^BSE500']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-03-06 -> 2025-10-27)')
2025-10-27 19:07:25 |